In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers datasets torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [4]:
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, Trainer, TrainingArguments
import torch


In [5]:
def load_data(src_file, tgt_file):
    with open(src_file, 'r', encoding='utf-8') as f:
        src_lines = f.readlines()
    with open(tgt_file, 'r', encoding='utf-8') as f:
        tgt_lines = f.readlines()

    src_lines = [line.strip() for line in src_lines]
    tgt_lines = [line.strip() for line in tgt_lines]

    return src_lines, tgt_lines

# Load the full dataset
src_file = '/content/drive/MyDrive/Language translator new/train.de'
tgt_file = '/content/drive/MyDrive/Language translator new/train.en'
src_lines, tgt_lines = load_data(src_file, tgt_file)

# Use only half of the data
num_samples = len(src_lines)
half_index = num_samples // 2
src_lines = src_lines[:half_index]
tgt_lines = tgt_lines[:half_index]

print(f"Using {len(src_lines)} samples for training.")


Using 14500 samples for training.


In [6]:
# Create a Dataset object
data_dict = {'de': src_lines, 'en': tgt_lines}
dataset = Dataset.from_dict(data_dict)

# Tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
# Tokenize the data
def tokenize_function(examples):
    src_encodings = tokenizer(examples['de'], padding='max_length', truncation=True, max_length=128)
    tgt_encodings = tokenizer(examples['en'], padding='max_length', truncation=True, max_length=128)

    return {'input_ids': src_encodings['input_ids'],
            'attention_mask': src_encodings['attention_mask'],
            'labels': tgt_encodings['input_ids']}

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Prepare data for training
data_collator = DataCollatorForSeq2Seq(tokenizer, model=None)

Map:   0%|          | 0/14500 [00:00<?, ? examples/s]

In [8]:
# Split the dataset into training and validation sets
train_dataset = tokenized_datasets.shuffle(seed=42).select([i for i in range(10000)])  # Adjust as needed
val_dataset = tokenized_datasets.shuffle(seed=42).select([i for i in range(1000)])

# Model
model = T5ForConditionalGeneration.from_pretrained('t5-small')


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [11]:
# Training Arguments
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    output_dir='./results',
    evaluation_strategy='epoch',
    num_train_epochs=1,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=10
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

In [12]:
# Translate function
def translate(model, tokenizer, text, max_length=128):
    model.eval()

    inputs = tokenizer(text, return_tensors="pt", max_length=max_length, truncation=True)
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)

    with torch.no_grad():
        outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length)

    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation

In [13]:
# Example usage
german_sentence = "Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche."
translated_text = translate(model, tokenizer, german_sentence)
print(f"German: {german_sentence}")
print(f"English: {translated_text}")

German: Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.
English: Two young white men are in the open in the near area of a large buffet.
